In [1]:
import requests
import datetime
import json

In [1]:
SUMMARY_INDEX = "gracc.osg.summary"
ENDPOINT = "https://gracc.opensciencegrid.org:443/q"
HEADERS = {'Content-Type': 'application/json'}

test_query = {
    "size": 0,
    "query": {
        "bool" : {
            "filter" : [
                { "term" : { "ResourceType": "Payload" }},
                { "range": {
                    "EndTime": {
                        "lte": int(datetime.datetime.now().timestamp()*1000),
                        "gte": int((datetime.datetime.now() - datetime.timedelta(days=365)).timestamp()*1000)
                    }
                }}
            ]
        }
    },
    "aggs": {
        "facilities": {
            "terms": {
                "field": "OIM_ProjectName",
                "size": 99999999
            },
            "aggs": {
                "facilityCpuProvided": {
                    "sum": {
                        "field": "CoreHours"
                    }
                },
                "facilityGpuProvided": {
                    "sum": {
                        "field": "GPUHours"
                    }
                },
                "countProjectsImpacted": {
                    "cardinality": {
                        "field": "ProjectName"
                    }
                },
                "countFieldsOfScienceImpacted": {
                    "cardinality": {
                        "field": "OIM_FieldOfScience"
                    }
                },
                "countOrganizationImpacted": {
                    "cardinality": {
                        "field": "OIM_Organization"
                    }
                },
                "gpu_bucket_filter": {
                  "bucket_selector": {
                    "buckets_path": {
                      "totalGPU": "facilityGpuProvided",
                      "totalCPU": "facilityCpuProvided"
                    },
                    "script": "params.totalGPU > 0 || params.totalCPU > 0"
                  }
                },
            }
        }
    }
}

response_json = requests.get(f"{ENDPOINT}/{SUMMARY_INDEX}/_search", headers=HEADERS).json()

response_json

NameError: name 'datetime' is not defined

In [11]:
json.dumps(test_query)

'{"size": 0, "query": {"bool": {"filter": [{"term": {"ResourceType": "Payload"}}, {"range": {"EndTime": {"lte": 1667862127510, "gte": 1636326127510}}}]}}, "aggs": {"facilities": {"terms": {"field": "OIM_Facility", "size": 99999999}, "aggs": {"facilityCpuProvided": {"sum": {"field": "CoreHours"}}, "facilityGpuProvided": {"sum": {"field": "GPUHours"}}, "countProjectsImpacted": {"cardinality": {"field": "ProjectName"}}, "countFieldsOfScienceImpacted": {"cardinality": {"field": "OIM_FieldOfScience"}}, "countOrganizationImpacted": {"cardinality": {"field": "OIM_Organization"}}, "gpu_bucket_filter": {"bucket_selector": {"buckets_path": {"totalGPU": "facilityGpuProvided", "totalCPU": "facilityCpuProvided"}, "script": "params.totalGPU > 0 || params.totalCPU > 0"}}}}}}'

In [17]:
gracc_facility_names = set(x['key'] for x in response_json['aggregations']['facilities']['buckets'])

In [18]:
topology_facility_names = json.load(open("/Users/clock/Downloads/facility_json.json")).keys()

In [19]:
gracc_facility_names.difference(topology_facility_names)

set()